Cài đặt thư viện

In [9]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import  GridSearchCV

Đọc dữ liệu

In [2]:
data = pd.read_csv('dataset.csv')

In [43]:
data.shape

(378199, 47)

In [34]:
data.head(10)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,0.00,46.216667,62.933333,0.450358,0.450358,0.0,0.0,0.0,0.0,...,21.453631,569.366667,8.364729e+07,9.5,33.826772,30.405360,2793.867241,0.166667,141.55,Mirai-greip_flood
1,0.000000,54.00,6.000000,67.820000,1.983954,1.983954,0.0,0.0,0.0,0.0,...,0.000000,54.000000,8.331409e+07,9.5,10.392305,0.000000,0.000000,0.000000,141.55,DDoS-PSHACK_Flood
2,0.006434,71.87,5.940000,62.960000,26.872619,26.872619,0.0,0.0,1.0,0.0,...,1.108627,54.490000,8.308969e+07,9.5,10.511199,1.569356,6.685512,0.190000,141.55,DDoS-SYN_Flood
3,2.376069,86.40,6.000000,64.000000,0.505044,0.505044,0.0,0.0,1.0,0.0,...,0.000000,54.000000,8.336141e+07,9.5,10.392305,0.000000,0.000000,0.000000,141.55,DDoS-SynonymousIP_Flood
4,0.000000,81.00,6.000000,64.000000,31.082580,31.082580,0.0,0.0,0.0,0.0,...,0.000000,54.000000,8.292580e+07,9.5,10.392305,0.000000,0.000000,0.000000,141.55,DoS-TCP_Flood
5,0.000000,54.00,6.000000,64.000000,0.390246,0.390246,0.0,0.0,0.0,0.0,...,0.000000,54.000000,8.333097e+07,9.5,10.392305,0.000000,0.000000,0.000000,141.55,DDoS-PSHACK_Flood
6,4.265998,108.00,6.000000,64.000000,0.468824,0.468824,0.0,0.0,1.0,0.0,...,0.000000,54.000000,8.298528e+07,9.5,10.392305,0.000000,0.000000,0.000000,141.55,DoS-SYN_Flood
7,9.360785,116.26,5.950000,63.850000,0.495251,0.495251,0.0,0.0,1.0,0.0,...,6.867622,55.990000,8.297355e+07,9.5,10.575351,9.727346,324.226415,0.270000,141.55,DoS-SYN_Flood
8,0.000000,54.00,6.000000,64.000000,7.937263,7.937263,0.0,0.0,0.0,0.0,...,0.000000,54.000000,8.331386e+07,9.5,10.392305,0.000000,0.000000,0.000000,141.55,DDoS-PSHACK_Flood
9,0.084924,35781.00,17.000000,64.000000,10079.497975,10079.497975,0.0,0.0,0.0,0.0,...,0.000000,50.000000,8.348742e+07,9.5,10.000000,0.000000,0.000000,0.000000,141.55,DDoS-UDP_Flood


In [3]:
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count',
       'syn_count', 'fin_count', 'urg_count', 'rst_count', 
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
       'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
       'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight', 
]
y_column = 'label'

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378199 entries, 0 to 378198
Data columns (total 47 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   flow_duration    378199 non-null  float64
 1   Header_Length    378199 non-null  float64
 2   Protocol Type    378199 non-null  float64
 3   Duration         378199 non-null  float64
 4   Rate             378199 non-null  float64
 5   Srate            378199 non-null  float64
 6   Drate            378199 non-null  float64
 7   fin_flag_number  378199 non-null  float64
 8   syn_flag_number  378199 non-null  float64
 9   rst_flag_number  378199 non-null  float64
 10  psh_flag_number  378199 non-null  float64
 11  ack_flag_number  378199 non-null  float64
 12  ece_flag_number  378199 non-null  float64
 13  cwr_flag_number  378199 non-null  float64
 14  ack_count        378199 non-null  float64
 15  syn_count        378199 non-null  float64
 16  fin_count        378199 non-null  floa

In [4]:
list_remove = ['Drate', 'ece_flag_number', 'cwr_flag_number', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'DHCP', 'ARP', 'IPv', 'LLC']

X_columns = list(filter(lambda x: x not in list_remove, X_columns))

In [39]:
data[y_column].value_counts()

Mirai-greip_flood          13004
BenignTraffic              13004
DictionaryBruteForce       13004
DDoS-HTTP_Flood            13004
VulnerabilityScan          13004
DDoS-SlowLoris             13004
Recon-PortScan             13004
Recon-OSScan               13004
DDoS-UDP_Fragmentation     13004
DDoS-ICMP_Fragmentation    13004
DoS-HTTP_Flood             13004
DDoS-PSHACK_Flood          13004
MITM-ArpSpoofing           13004
Mirai-greeth_flood         13004
DDoS-ACK_Fragmentation     13004
DNS_Spoofing               13004
DoS-UDP_Flood              13004
DDoS-TCP_Flood             13004
Recon-HostDiscovery        13004
DDoS-ICMP_Flood            13004
DDoS-RSTFINFlood           13004
DDoS-UDP_Flood             13004
DoS-SYN_Flood              13004
Mirai-udpplain             13004
DoS-TCP_Flood              13004
DDoS-SynonymousIP_Flood    13004
DDoS-SYN_Flood             13004
BrowserHijacking            5859
CommandInjection            5409
SqlInjection                5245
XSS       

Phân tích từng cột

In [40]:
data.describe()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
count,378199.000000,3.781990e+05,378199.000000,378199.000000,3.781990e+05,3.781990e+05,3.781990e+05,378199.000000,378199.000000,378199.000000,...,378199.000000,378199.000000,378199.000000,3.781990e+05,378199.000000,378199.000000,378199.000000,3.781990e+05,378199.000000,378199.000000
mean,97.081915,2.347648e+05,10.698250,76.981834,6.021942e+03,6.021942e+03,8.683915e-07,0.034979,0.131304,0.057163,...,339.526437,177.722510,339.559351,8.327897e+07,9.496680,21.543326,251.174798,1.478965e+05,0.491403,141.467052
std,1224.300103,8.147990e+05,10.506249,30.178793,7.066490e+04,7.066490e+04,2.508006e-04,0.183727,0.337733,0.232154,...,437.731760,306.771560,438.858048,4.663730e+07,2.238269,13.967703,433.903636,5.478475e+05,0.439761,57.632531
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,42.000000,0.000000,42.000000,0.000000e+00,1.000000,9.165151,0.000000,0.000000e+00,0.000000,1.000000
25%,0.000000,5.400000e+01,6.000000,64.000000,3.139673e+00,3.139673e+00,0.000000e+00,0.000000,0.000000,0.000000,...,54.000000,0.000000,54.000000,8.300270e+07,9.500000,10.392305,0.000000,0.000000e+00,0.000000,141.550000
50%,0.469855,1.094460e+03,6.060000,64.000000,2.119618e+01,2.119618e+01,0.000000e+00,0.000000,0.000000,0.000000,...,107.457379,23.586296,106.400000,8.325418e+07,9.500000,14.498068,33.222164,1.534253e+03,0.540000,141.550000
75%,11.799132,3.570771e+04,12.000000,75.460000,1.130005e+02,1.130005e+02,0.000000e+00,0.000000,0.000000,0.000000,...,568.729273,245.114824,569.665000,8.358197e+07,9.500000,33.286634,345.751278,1.144262e+05,0.950000,141.550000
max,99685.656389,9.887854e+06,47.000000,255.000000,6.291456e+06,6.291456e+06,1.152843e-01,1.000000,1.000000,1.000000,...,8210.884444,8026.920297,6730.300000,1.676394e+08,15.000000,127.048500,11351.779549,7.370417e+07,1.000000,244.600000


Hyperparameter Tuning

In [ ]:
par = {
    'n_estimators': [100, 500],
    'max_depth': [10, 30, 50]
}

grid = GridSearchCV(RandomForestClassifier(), par)
grid.fit(data[X_columns],data[y_column])

print(grid.best_params_)

Chia dữ liệu

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data[X_columns], data[y_column], test_size=0.2, random_state=42)

Tạo model

In [11]:
rf_model = RandomForestClassifier(random_state=42, max_depth=50, n_estimators=500)

In [ ]:
score_rf = cross_val_score(rf_model, X_train, y_train, cv = 5)

In [13]:
print(score_rf)
print("Arg: ", np.average(score_rf))

[0.92277565 0.9227426  0.9222303  0.92219725 0.92280742]
Arg:  0.9225506438637741


Train model và Dự đoán

In [44]:
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9252247488101534
Classification Report:
                         precision    recall  f1-score   support

       Backdoor_Malware       0.75      0.57      0.65       639
          BenignTraffic       0.79      0.87      0.82      2627
       BrowserHijacking       0.85      0.72      0.78      1173
       CommandInjection       0.71      0.71      0.71      1066
 DDoS-ACK_Fragmentation       1.00      0.99      1.00      2584
        DDoS-HTTP_Flood       1.00      1.00      1.00      2560
        DDoS-ICMP_Flood       1.00      1.00      1.00      2590
DDoS-ICMP_Fragmentation       1.00      0.99      1.00      2637
      DDoS-PSHACK_Flood       1.00      1.00      1.00      2643
       DDoS-RSTFINFlood       1.00      1.00      1.00      2569
         DDoS-SYN_Flood       1.00      1.00      1.00      2557
         DDoS-SlowLoris       0.99      1.00      0.99      2661
DDoS-SynonymousIP_Flood       1.00      1.00      1.00      2596
         DDoS-TCP_Flood       1.00   

Lưu model

In [47]:
pickle.dump(rf_model, open('C:/Users/vuhuy/Downloads/IOT/model_train_history/model_34class.pickle', 'wb'))

In [48]:
dict_7classes = {}
dict_7classes['DDoS-RSTFINFlood'] = 'DDoS'
dict_7classes['DDoS-PSHACK_Flood'] = 'DDoS'
dict_7classes['DDoS-SYN_Flood'] = 'DDoS'
dict_7classes['DDoS-UDP_Flood'] = 'DDoS'
dict_7classes['DDoS-TCP_Flood'] = 'DDoS'
dict_7classes['DDoS-ICMP_Flood'] = 'DDoS'
dict_7classes['DDoS-SynonymousIP_Flood'] = 'DDoS'
dict_7classes['DDoS-ACK_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-UDP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-ICMP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-SlowLoris'] = 'DDoS'
dict_7classes['DDoS-HTTP_Flood'] = 'DDoS'

dict_7classes['DoS-UDP_Flood'] = 'DoS'
dict_7classes['DoS-SYN_Flood'] = 'DoS'
dict_7classes['DoS-TCP_Flood'] = 'DoS'
dict_7classes['DoS-HTTP_Flood'] = 'DoS'


dict_7classes['Mirai-greeth_flood'] = 'Mirai'
dict_7classes['Mirai-greip_flood'] = 'Mirai'
dict_7classes['Mirai-udpplain'] = 'Mirai'

dict_7classes['Recon-PingSweep'] = 'Recon'
dict_7classes['Recon-OSScan'] = 'Recon'
dict_7classes['Recon-PortScan'] = 'Recon'
dict_7classes['VulnerabilityScan'] = 'Recon'
dict_7classes['Recon-HostDiscovery'] = 'Recon'

dict_7classes['DNS_Spoofing'] = 'Spoofing'
dict_7classes['MITM-ArpSpoofing'] = 'Spoofing'

dict_7classes['BenignTraffic'] = 'Benign'

dict_7classes['BrowserHijacking'] = 'Web'
dict_7classes['Backdoor_Malware'] = 'Web'
dict_7classes['XSS'] = 'Web'
dict_7classes['Uploading_Attack'] = 'Web'
dict_7classes['SqlInjection'] = 'Web'
dict_7classes['CommandInjection'] = 'Web'


dict_7classes['DictionaryBruteForce'] = 'BruteForce'

In [49]:
new_y = [dict_7classes[k] for k in data[y_column]]
data[y_column] = new_y

In [50]:
data[y_column].value_counts()

DDoS          156048
Recon          54278
DoS            52016
Mirai          39012
Spoofing       26008
Web            24829
Benign         13004
BruteForce     13004
Name: label, dtype: int64

In [51]:
X_train, X_test, y_train, y_test = train_test_split(data[X_columns], data[y_column], test_size=0.2, random_state=42)

rf_model1 = RandomForestClassifier(random_state=42)

rf_model1.fit(X_train, y_train)

y_pred = rf_model1.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9486250661025912
Classification Report:
              precision    recall  f1-score   support

      Benign       0.85      0.80      0.82      2627
  BruteForce       0.87      0.72      0.79      2624
        DDoS       1.00      1.00      1.00     31312
         DoS       1.00      1.00      1.00     10294
       Mirai       1.00      1.00      1.00      7856
       Recon       0.86      0.92      0.89     10822
    Spoofing       0.91      0.82      0.86      5167
         Web       0.79      0.85      0.82      4938

    accuracy                           0.95     75640
   macro avg       0.91      0.89      0.90     75640
weighted avg       0.95      0.95      0.95     75640



Lưu model

In [52]:
pickle.dump(rf_model1, open('C:/Users/vuhuy/Downloads/IOT/model_train_history/model_7class.pickle', 'wb'))

Load model

In [53]:
load_model = pickle.load(open('model_train_history/model_34class.pickle', 'rb'))

In [54]:
print(load_model)

RandomForestClassifier(max_depth=50, n_estimators=500, random_state=42)


In [55]:
data_test = pd.read_csv('data.csv')

In [56]:
y_pred = rf_model.predict(data_test[X_columns])

In [57]:
accuracy = accuracy_score(data_test[y_column], y_pred)
print(f'Accuracy: {accuracy}')

print("Classification Report:")
print(classification_report(data_test[y_column], y_pred))

Accuracy: 1.0
Classification Report:
                         precision    recall  f1-score   support

 DDoS-ACK_Fragmentation       1.00      1.00      1.00         1
        DDoS-ICMP_Flood       1.00      1.00      1.00         2
       DDoS-RSTFINFlood       1.00      1.00      1.00         2
DDoS-SynonymousIP_Flood       1.00      1.00      1.00         1
         DDoS-TCP_Flood       1.00      1.00      1.00         1
         DDoS-UDP_Flood       1.00      1.00      1.00         3
          DoS-UDP_Flood       1.00      1.00      1.00         2

               accuracy                           1.00        12
              macro avg       1.00      1.00      1.00        12
           weighted avg       1.00      1.00      1.00        12

